#### Importing and Combining Data

In [1]:
# Point to the match name sub-folder
match_name="LAN vs SUR - 2023-07-07"

In [2]:
import pandas as pd

# Point to the location of extracted csvs
team1_data=pd.read_csv("Scraped Data/LAN vs SUR - 2023-07-07/2023-07-07-LAN-Batting.csv")
team2_data=pd.read_csv("Scraped Data/LAN vs SUR - 2023-07-07/2023-07-07-SUR-Batting.csv")

In [3]:
team1_data['Team'] = "LAN"
team2_data['Team'] = "SUR"

In [4]:
print(team1_data.head())
print(team1_data.shape)
print(team2_data.shape)

   ball score                  commentary Team
0  19.6     1  Abbott to Mitchell,  1 run  LAN
1  19.5     1     Abbott to Jones,  1 run  LAN
2  19.4     1  Abbott to Mitchell,  1 run  LAN
3  19.3     •  Abbott to Mitchell, no run  LAN
4  19.2     •  Abbott to Mitchell, no run  LAN
(125, 4)
(124, 4)


In [5]:
#Sorting the two datasets before union- extracted in reverse order so just reversed it
team1_data = team1_data.iloc[::-1].reset_index(drop=True)
team2_data = team2_data.iloc[::-1].reset_index(drop=True)

In [6]:
#Union 2 Datasets together
data = pd.concat([team1_data, team2_data], ignore_index=True)

In [7]:
data.shape

(249, 4)

In [8]:
data.head()

,ball,score,commentary,Team
0,0.1,1,"Jacks to Salt, 1 run",LAN
1,0.2,1,"Jacks to Buttler, 1 run",LAN
2,0.3,4,"Jacks to Salt, FOUR runs",LAN
3,0.4,1,"Jacks to Salt, 1 run",LAN
4,0.5,6,"Jacks to Buttler, SIX runs",LAN


#### Cleaning Steps

In [9]:
data["score"].unique()

array(['1', '4', '6', '•', '4lb', 'W', '2', '4b', '3lb', '3n-l', '1lb',
       '1w', '1W', '3nb', '2nb', '6nb', '3'], dtype=object)

In [10]:
#Replace "." with "0"
data['score'] = data['score'].replace("•", 0)

In [11]:
#Extracting Runs or Numeric elements from the Scores column
data["Runs"]=None
data['Runs'] = data['score'].astype(str).str.extract('(\d+)')
data['Runs'] = data['Runs'].fillna(0).astype(int)

<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
/var/folders/0r/jdkpl6hs7m184t4p3zcgbsv00000gn/T/ipykernel_9694/742093441.py:3: SyntaxWarning: invalid escape sequence '\d'
  data['Runs'] = data['score'].astype(str).str.extract('(\d+)')


In [12]:
data.groupby('Team')['Runs'].sum()

Team
LAN    174
SUR    187
Name: Runs, dtype: int64

In [13]:
# Create 'Wickets' column: 1 if OUT, 0 otherwise
data['Wickets'] = data['score'].apply(lambda x: 1 if 'W' in str(x) else 0)

In [14]:
data.groupby('Team')['Wickets'].sum()

Team
LAN    6
SUR    5
Name: Wickets, dtype: int64

In [15]:
data

,ball,score,commentary,Team,Runs,Wickets
0,0.1,1,"Jacks to Salt, 1 run",LAN,1,0
1,0.2,1,"Jacks to Buttler, 1 run",LAN,1,0
2,0.3,4,"Jacks to Salt, FOUR runs",LAN,4,0
3,0.4,1,"Jacks to Salt, 1 run",LAN,1,0
4,0.5,6,"Jacks to Buttler, SIX runs",LAN,6,0
...,...,...,...,...,...,...
244,19.2,1,"Bailey to Narine, 1 run",SUR,1,0
245,19.3,1,"Bailey to Jordan, 1 run",SUR,1,0
246,19.4,1,"Bailey to Narine, 1 run",SUR,1,0
247,19.5,6,"Bailey to Jordan, SIX runs",SUR,6,0


In [16]:
data['Ball ID'] = data.groupby('Team').cumcount()

In [17]:
data.tail()

,ball,score,commentary,Team,Runs,Wickets,Ball ID
244,19.2,1,"Bailey to Narine, 1 run",SUR,1,0,119
245,19.3,1,"Bailey to Jordan, 1 run",SUR,1,0,120
246,19.4,1,"Bailey to Narine, 1 run",SUR,1,0,121
247,19.5,6,"Bailey to Jordan, SIX runs",SUR,6,0,122
248,19.6,2,"Bailey to Jordan, 2 runs",SUR,2,0,123


In [18]:
# convert the 'ball' column to a string
data['ball'] = data['ball'].astype(str)

# Split 'ball' into 'Over' and 'Ball' columns
data[['Over', 'Ball']] = data['ball'].str.split('.', expand=True)

# Convert 'Over' and 'Ball' columns to integers
data['Over'] = data['Over'].astype(int)
data['Ball'] = data['Ball'].astype(int)

In [19]:
data.head(2)

,ball,score,commentary,Team,Runs,Wickets,Ball ID,Over,Ball
0,0.1,1,"Jacks to Salt, 1 run",LAN,1,0,0,0,1
1,0.2,1,"Jacks to Buttler, 1 run",LAN,1,0,1,0,2


In [20]:
#Keep relevant columns only
data = data[['Team', 'Over', 'Ball','Ball ID','Runs','Wickets']]

In [21]:
print(data.head())
print(data.shape)

  Team  Over  Ball  Ball ID  Runs  Wickets
0  LAN     0     1        0     1        0
1  LAN     0     2        1     1        0
2  LAN     0     3        2     4        0
3  LAN     0     4        3     1        0
4  LAN     0     5        4     6        0
(249, 6)


In [24]:
data.to_csv(f"Scraped Data/{match_name}/cleaned_data.csv", index=False)